In [43]:
import shutil
from pathlib import Path

import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, FactorRange, Whisker, Legend
from bokeh.palettes import Bright6, Category10 
from bokeh.transform import factor_cmap, dodge
from bokeh.io import output_notebook
from bokeh.io import export_svgs, export_png

from dctap.libs.pandas import set_defaultoptions, display, displaydf_full
from dctap.qpcr.constants import QPCRPATHS
from dctap.qpcr.utils.core import *

In [44]:
# Set pandas settings
set_defaultoptions(pd, supresscopywarning=None)
output_notebook()

Loading BokehJS ...

In [45]:
# Read and annotate data from different plates and combine them
experiment_id = "JR98-250306"
plate_ids = ["JR98-250306-plate1", "JR98-250306-plate2"]

dfs = []
for plate_id in plate_ids:
    dfs.append(get_plate_data(experiment_id, plate_id))

df = pd.concat(dfs)
df.reset_index(inplace=True, drop=True)

display(df)

,experiment_id,Well,Sample,Primer,Cq,plate_id
0,JR98-250306,A01,Unperturbed_NaN_ctrl_D0_A,GAPDH,14.803,JR98-250306-plate1
1,JR98-250306,A02,Unperturbed_NaN_ctrl_D0_A,GAPDH,14.216,JR98-250306-plate1
2,JR98-250306,A03,Unperturbed_NaN_ctrl_D0_B,GAPDH,14.903,JR98-250306-plate1
3,JR98-250306,A04,Unperturbed_NaN_ctrl_D0_B,GAPDH,15.045,JR98-250306-plate1
4,JR98-250306,A05,Unperturbed_NaN_ctrl_D2-2DD_A,GAPDH,14.979,JR98-250306-plate1
...,...,...,...,...,...,...
379,JR98-250306,L08,Perturbed_R2_150vv_D0-2DD_A,CYP26A1,29.170,JR98-250306-plate2
380,JR98-250306,L09,Perturbed_R2_150vv_D0-2DD_B,CYP26A1,26.397,JR98-250306-plate2
381,JR98-250306,L10,Perturbed_R2_150vv_D0-2DD_B,CYP26A1,27.560,JR98-250306-plate2
382,JR98-250306,L11,Perturbed_R2_150vv_D2-2DD_NaN,CYP26A1,21.802,JR98-250306-plate2


In [46]:
df_samples = get_sample_metadata(cast(pd.Series, df.Sample), sep="_")
with displaydf_full():
  display(df_samples)

,Sample,0,1,2,3,4
0,Unperturbed_NaN_ctrl_D0_A,Unperturbed,NaN,ctrl,D0,A
1,Unperturbed_NaN_ctrl_D0_B,Unperturbed,NaN,ctrl,D0,B
2,Unperturbed_NaN_ctrl_D2-2DD_A,Unperturbed,NaN,ctrl,D2-2DD,A
3,Unperturbed_NaN_ctrl_D2-2DD_B,Unperturbed,NaN,ctrl,D2-2DD,B
4,Perturbed_R1_100vv_D0-2DD_A,Perturbed,R1,100vv,D0-2DD,A
5,Perturbed_R1_100vv_D0-2DD_B,Perturbed,R1,100vv,D0-2DD,B
6,Perturbed_R1_100vv_D2-2DD_NaN,Perturbed,R1,100vv,D2-2DD,NaN
7,Perturbed_R1_150vv_D0-2DD_A,Perturbed,R1,150vv,D0-2DD,A
8,Perturbed_R1_150vv_D0-2DD_B,Perturbed,R1,150vv,D0-2DD,B
9,Perturbed_R1_150vv_D2-2DD_NaN,Perturbed,R1,150vv,D2-2DD,NaN


In [47]:
conditions = ["bio_reps", "ctrl_calibrator", "cond_diff"]
df = set_conditions(
    df,
    df_samples,
    conditions=conditions,
    merge_cols=["0123", "0123", "012"],
)

display(df)

,experiment_id,Well,Sample,Primer,Cq,plate_id,bio_reps,ctrl_calibrator,cond_diff
0,JR98-250306,A01,Unperturbed_NaN_ctrl_D0_A,GAPDH,14.803,JR98-250306-plate1,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl
1,JR98-250306,A02,Unperturbed_NaN_ctrl_D0_A,GAPDH,14.216,JR98-250306-plate1,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl
2,JR98-250306,A03,Unperturbed_NaN_ctrl_D0_B,GAPDH,14.903,JR98-250306-plate1,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl
3,JR98-250306,A04,Unperturbed_NaN_ctrl_D0_B,GAPDH,15.045,JR98-250306-plate1,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl
4,JR98-250306,A05,Unperturbed_NaN_ctrl_D2-2DD_A,GAPDH,14.979,JR98-250306-plate1,Unperturbed_NaN_ctrl_D2-2DD,Unperturbed_NaN_ctrl_D2-2DD,Unperturbed_NaN_ctrl
...,...,...,...,...,...,...,...,...,...
379,JR98-250306,L08,Perturbed_R2_150vv_D0-2DD_A,CYP26A1,29.170,JR98-250306-plate2,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv
380,JR98-250306,L09,Perturbed_R2_150vv_D0-2DD_B,CYP26A1,26.397,JR98-250306-plate2,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv
381,JR98-250306,L10,Perturbed_R2_150vv_D0-2DD_B,CYP26A1,27.560,JR98-250306-plate2,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv
382,JR98-250306,L11,Perturbed_R2_150vv_D2-2DD_NaN,CYP26A1,21.802,JR98-250306-plate2,Perturbed_R2_150vv_D2-2DD,Perturbed_R2_150vv_D2-2DD,Perturbed_R2_150vv


In [48]:
df1 = get_deltaCq_expression_bulkdata(
    df,
    ref_primer="GAPDH",
    test_primers=get_primers(df),
    drop_customcols=conditions,
)
display(df1)

,Sample,bio_reps,ctrl_calibrator,cond_diff,Cq_ref_GAPDH_mean,Cq_ref_GAPDH_std,Cq_ref_GAPDH_ste,Cq_test_OCT4_mean,Cq_test_OCT4_std,Cq_test_OCT4_ste,deltaCq_OCT4vGAPDH,Cq_test_NANOG_mean,Cq_test_NANOG_std,Cq_test_NANOG_ste,deltaCq_NANOGvGAPDH,Cq_test_CER1_mean,Cq_test_CER1_std,Cq_test_CER1_ste,deltaCq_CER1vGAPDH,Cq_test_LHX1_mean,Cq_test_LHX1_std,Cq_test_LHX1_ste,deltaCq_LHX1vGAPDH,Cq_test_CYP26A1_mean,Cq_test_CYP26A1_std,Cq_test_CYP26A1_ste,deltaCq_CYP26A1vGAPDH
0,Unperturbed_NaN_ctrl_D0_A,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl,14.63875,0.287111,0.143556,15.07400,0.288531,0.144265,0.43525,16.13400,0.132156,0.066078,1.49525,21.12375,0.221089,0.110544,6.48500,24.59875,1.318976,0.659488,9.96000,25.51850,0.769531,0.384765,10.87975
1,Unperturbed_NaN_ctrl_D0_B,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl,15.20675,0.299507,0.149753,15.56025,0.204678,0.102339,0.35350,17.08000,0.116112,0.058056,1.87325,22.07500,0.392894,0.196447,6.86825,25.91175,1.828875,0.914437,10.70500,26.69025,0.532673,0.266336,11.48350
2,Unperturbed_NaN_ctrl_D2-2DD_A,Unperturbed_NaN_ctrl_D2-2DD,Unperturbed_NaN_ctrl_D2-2DD,Unperturbed_NaN_ctrl,15.12400,0.294067,0.147033,15.53200,0.111221,0.055610,0.40800,17.88750,0.296853,0.148426,2.76350,17.90425,0.296904,0.148452,2.78025,21.52725,0.318859,0.159429,6.40325,19.42850,0.842046,0.421023,4.30450
3,Unperturbed_NaN_ctrl_D2-2DD_B,Unperturbed_NaN_ctrl_D2-2DD,Unperturbed_NaN_ctrl_D2-2DD,Unperturbed_NaN_ctrl,15.24875,0.185007,0.092503,15.80350,0.410667,0.205333,0.55475,18.15800,0.586626,0.293313,2.90925,18.73150,0.483215,0.241608,3.48275,21.85125,0.527771,0.263886,6.60250,18.19600,3.015529,1.507765,2.94725
4,Perturbed_R1_100vv_D0-2DD_A,Perturbed_R1_100vv_D0-2DD,Perturbed_R1_100vv_D0-2DD,Perturbed_R1_100vv,14.47125,0.149876,0.074938,14.96725,0.165792,0.082896,0.49600,16.82750,0.241937,0.120969,2.35625,21.89625,0.163665,0.081833,7.42500,27.16325,2.157864,1.078932,12.69200,27.66450,1.221131,0.610566,13.19325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,Perturbed_R2_100vv_D0-2DD_B,Perturbed_R2_100vv_D0-2DD,Perturbed_R2_100vv_D0-2DD,Perturbed_R2_100vv,16.04500,0.160460,0.080230,16.35725,0.151280,0.075640,0.31225,18.16750,0.091420,0.045710,2.12250,23.67475,0.130760,0.065380,7.62975,27.08725,0.162637,0.081319,11.04225,26.84600,0.273311,0.136655,10.80100
12,Perturbed_R2_100vv_D2-2DD_NaN,Perturbed_R2_100vv_D2-2DD,Perturbed_R2_100vv_D2-2DD,Perturbed_R2_100vv,16.89675,0.138947,0.069473,17.75575,0.140488,0.070244,0.85900,20.28425,0.066173,0.033087,3.38750,21.77675,0.053450,0.026725,4.88000,26.23350,0.244626,0.122313,9.33675,21.35375,0.926729,0.463364,4.45700
13,Perturbed_R2_150vv_D0-2DD_A,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv,15.95975,0.174299,0.087150,16.00975,0.179307,0.089653,0.05000,17.74275,0.173997,0.086998,1.78300,23.19200,0.490071,0.245036,7.23225,27.25350,0.421918,0.210959,11.29375,28.33375,1.014165,0.507083,12.37400
14,Perturbed_R2_150vv_D0-2DD_B,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv,16.09675,0.165532,0.082766,16.05000,0.042198,0.021099,-0.04675,17.95325,0.133642,0.066821,1.85650,24.56200,0.203252,0.101626,8.46525,27.20350,0.244148,0.122074,11.10675,26.84975,0.504996,0.252498,10.75300


In [49]:
df2 = get_deltaCq_stats(df1, biorep_col="bio_reps")
df_calibrators = get_calibrators(
    df2,
    ctrl_col="ctrl_calibrator",
    condition_col="cond_diff",
    assign_ctrl_samples=[
        "Unperturbed_NaN_ctrl_D0",
        "Perturbed_R1_100vv_D0-2DD",
        "Perturbed_R1_150vv_D0-2DD",
        "Perturbed_R2_100vv_D0-2DD",
        "Perturbed_R2_150vv_D0-2DD",
    ],
    assign_cond_group=[
        "Unperturbed_NaN_ctrl",
        "Perturbed_R1_100vv",
        "Perturbed_R1_150vv",
        "Perturbed_R2_100vv",
        "Perturbed_R2_150vv",
    ],
)
with displaydf_full():
  display(df_calibrators)

,deltaCq_OCT4vGAPDH,deltaCq_NANOGvGAPDH,deltaCq_CER1vGAPDH,deltaCq_LHX1vGAPDH,deltaCq_CYP26A1vGAPDH
Unperturbed_NaN_ctrl,0.394375,1.684250,6.676625,10.332500,11.181625
Perturbed_R1_100vv,0.395875,2.265250,7.739250,12.582250,12.675500
Perturbed_R1_150vv,-0.088750,1.775625,7.435500,11.747000,11.131250
Perturbed_R2_100vv,0.195000,1.632875,6.965375,10.640875,10.861000
Perturbed_R2_150vv,0.001625,1.819750,7.848750,11.200250,11.563500


In [50]:
df3 = get_deltadeltaCqMethod_foldchange(
    df1, df_calibrators, biorep_col="bio_reps", condition_col="cond_diff"
)

with displaydf_full():
  display(df3)

,bio_reps,ctrl_calibrator,cond_diff,2^(deltadeltaCq_OCT4vGAPDH)_mean,2^(deltadeltaCq_OCT4vGAPDH)_std,2^(deltadeltaCq_OCT4vGAPDH)_ste,2^(deltadeltaCq_OCT4vGAPDH)_ci95_upper,2^(deltadeltaCq_OCT4vGAPDH)_ci95_lower,2^(deltadeltaCq_NANOGvGAPDH)_mean,2^(deltadeltaCq_NANOGvGAPDH)_std,2^(deltadeltaCq_NANOGvGAPDH)_ste,2^(deltadeltaCq_NANOGvGAPDH)_ci95_upper,2^(deltadeltaCq_NANOGvGAPDH)_ci95_lower,2^(deltadeltaCq_CER1vGAPDH)_mean,2^(deltadeltaCq_CER1vGAPDH)_std,2^(deltadeltaCq_CER1vGAPDH)_ste,2^(deltadeltaCq_CER1vGAPDH)_ci95_upper,2^(deltadeltaCq_CER1vGAPDH)_ci95_lower,2^(deltadeltaCq_LHX1vGAPDH)_mean,2^(deltadeltaCq_LHX1vGAPDH)_std,2^(deltadeltaCq_LHX1vGAPDH)_ste,2^(deltadeltaCq_LHX1vGAPDH)_ci95_upper,2^(deltadeltaCq_LHX1vGAPDH)_ci95_lower,2^(deltadeltaCq_CYP26A1vGAPDH)_mean,2^(deltadeltaCq_CYP26A1vGAPDH)_std,2^(deltadeltaCq_CYP26A1vGAPDH)_ste,2^(deltadeltaCq_CYP26A1vGAPDH)_ci95_upper,2^(deltadeltaCq_CYP26A1vGAPDH)_ci95_lower
0,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl_D0,Unperturbed_NaN_ctrl,1.000401,0.040073,0.028336,1.055940,0.944862,1.008593,0.185799,0.131380,1.266098,0.751089,1.008834,0.188395,0.133215,1.269936,0.747732,1.033519,0.369217,0.261076,1.545227,0.521810,1.021971,0.298080,0.210774,1.435089,0.608854
1,Unperturbed_NaN_ctrl_D2-2DD,Unperturbed_NaN_ctrl_D2-2DD,Unperturbed_NaN_ctrl,0.942696,0.067746,0.047904,1.036588,0.848805,0.450536,0.032157,0.022739,0.495104,0.405968,12.020860,4.059083,2.870205,17.646461,6.395258,14.251699,1.389587,0.982587,16.177569,12.325829,209.353582,129.830530,91.804048,389.289516,29.417647
2,Perturbed_R1_100vv_D0-2DD,Perturbed_R1_100vv_D0-2DD,Perturbed_R1_100vv,1.002409,0.098227,0.069457,1.138545,0.866273,1.001990,0.089263,0.063118,1.125702,0.878278,1.023817,0.310488,0.219548,1.454131,0.593503,1.002895,0.107687,0.076146,1.152142,0.853648,1.065090,0.518493,0.366630,1.783686,0.346495
3,Perturbed_R1_100vv_D2-2DD,Perturbed_R1_100vv_D2-2DD,Perturbed_R1_100vv,0.434785,NaN,NaN,NaN,NaN,0.119245,NaN,NaN,NaN,NaN,1.954094,NaN,NaN,NaN,NaN,3.750284,NaN,NaN,NaN,NaN,211.607947,NaN,NaN,NaN,NaN
4,Perturbed_R1_150vv_D0-2DD,Perturbed_R1_150vv_D0-2DD,Perturbed_R1_150vv,1.000282,0.033577,0.023743,1.046817,0.953746,1.000965,0.062144,0.043942,1.087092,0.914838,1.003015,0.109899,0.077710,1.155327,0.850702,1.211495,0.967183,0.683901,2.551942,-0.128951,1.960061,2.384047,1.685776,5.264182,-1.344059
5,Perturbed_R1_150vv_D2-2DD,Perturbed_R1_150vv_D2-2DD,Perturbed_R1_150vv,0.394063,NaN,NaN,NaN,NaN,0.156109,NaN,NaN,NaN,NaN,5.817906,NaN,NaN,NaN,NaN,5.558710,NaN,NaN,NaN,NaN,455.639812,NaN,NaN,NaN,NaN
6,Perturbed_R2_100vv_D0-2DD,Perturbed_R2_100vv_D0-2DD,Perturbed_R2_100vv,1.003304,0.115062,0.081361,1.162772,0.843837,1.058145,0.489226,0.345935,1.736177,0.380113,1.107922,0.674523,0.476960,2.042763,0.173081,1.038951,0.398546,0.281815,1.591308,0.486594,1.000865,0.058832,0.041601,1.082403,0.919327
7,Perturbed_R2_100vv_D2-2DD,Perturbed_R2_100vv_D2-2DD,Perturbed_R2_100vv,0.631126,NaN,NaN,NaN,NaN,0.296350,NaN,NaN,NaN,NaN,4.243854,NaN,NaN,NaN,NaN,2.469339,NaN,NaN,NaN,NaN,84.682972,NaN,NaN,NaN,NaN
8,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv_D0-2DD,Perturbed_R2_150vv,1.000562,0.047429,0.033537,1.066295,0.934829,1.000324,0.036028,0.025476,1.050257,0.950392,1.092701,0.622890,0.440450,1.955983,0.229419,1.002101,0.091718,0.064855,1.129216,0.874986,1.162002,0.836956,0.591817,2.321964,0.002039
9,Perturbed_R2_150vv_D2-2DD,Perturbed_R2_150vv_D2-2DD,Perturbed_R2_150vv,0.757661,NaN,NaN,NaN,NaN,0.488862,NaN,NaN,NaN,NaN,10.356754,NaN,NaN,NaN,NaN,7.505770,NaN,NaN,NaN,NaN,118.049654,NaN,NaN,NaN,NaN


In [51]:
def rename_col(col_name: str) -> str:
    pattern = r"^2\^\(.*_(.*?)v.*\)_(.*)$"
    match = re.match(pattern, col_name)
    if match:
        return f"{match.group(1)}_{match.group(2)}"
    else:
        return col_name

In [54]:
# Renaming cols
df = df3.copy()
df = df.rename(columns=rename_col)

# Extracting foldchange and cleaning up column names
foldchange_pattern = r".*_mean"
diffs_ori = ["D0", "D2", "R1 100vv D0", "R1 100vv D2", "R1 150vv D0", "R1 150vv D2", "R2 100vv D0", "R2 100vv D2", "R2 150vv D0", "R2 150vv D2"]
primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

ci95_upper_pattern = r".*_ci95_upper"
ci95_lower_pattern = r".*_ci95_lower"
ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]


primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

data = {
    "CHIR Timing": diffs_ori,
    "CER1": df[primers[2]],
    "CER1_upper": df[ci95_upper[2]],
    "CER1_lower": df[ci95_lower[2]],
    "LHX1": df[primers[3]],
    "LHX1_upper": df[ci95_upper[3]],
    "LHX1_lower": df[ci95_lower[3]],
    "CYP26A": df[primers[4]],
    "CYP26A_upper": df[ci95_upper[4]],
    "CYP26A_lower": df[ci95_lower[4]],
}
primers = [re.sub(r"_mean$", "", s) for s in primers]
source = ColumnDataSource(data=data)

p = figure(
    x_range=diffs_ori, y_range=(0.001, 10000), height=350, width=650,
    title="Relative fold change of DE marker genes (vs GAPDH) in WTC11 derived samples",
    toolbar_location=None, tools="", y_axis_type="log"
)

bar_width = 0.2
p.vbar(
    x=dodge("CHIR Timing", -0.3, range=p.x_range),
    top="CER1", bottom=0.1, width=bar_width, source=source,
    color=Bright6[0], legend_label="CER1"
)
p.vbar(
    x=dodge("CHIR Timing", -0.1, range=p.x_range),
    top="LHX1", bottom=0.1, width=bar_width, source=source,
    color=Bright6[1], legend_label="LHX1"
)

p.vbar(
    x=dodge("CHIR Timing", 0.1, range=p.x_range),
    top="CYP26A", bottom=0.1, width=bar_width, source=source,
    color=Bright6[2], legend_label="CYP26A"
)

cer1_err = Whisker(
    base=dodge("CHIR Timing", -0.3, range=p.x_range),
    upper='CER1_upper', lower='CER1_lower', source=source, level="overlay"
)
cer1_err.upper_head.size = 10
cer1_err.lower_head.size = 10
p.add_layout(cer1_err)

lhx1_err = Whisker(
    base=dodge("CHIR Timing", -0.1, range=p.x_range),
    upper='LHX1_upper', lower='LHX1_lower', source=source, level="overlay"
)
lhx1_err.upper_head.size = 10
lhx1_err.lower_head.size = 10
p.add_layout(lhx1_err)


cyp_err = Whisker(
    base=dodge("CHIR Timing", 0.1, range=p.x_range),
    upper='CYP26A_upper', lower='CYP26A_lower', source=source, level="overlay"
)
cyp_err.upper_head.size = 10
cyp_err.lower_head.size = 10
p.add_layout(cyp_err)

p.y_range.start = 0.1
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "horizontal"
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label = "iPSC to DE Differentiation Sample Days (Unperturbed vs Infected)"
p.yaxis.axis_label = "Relative fold change wrt D0 (2^(-ΔΔCq))" 


# Export directory
exportfilename = "JR98-250306_qpcr_r1_d2markers.png"
source = QPCRPATHS.ROOT/ "analysis" / exportfilename
destination = QPCRPATHS.DATADIR / experiment_id / "250306_qpcr_analysis" / "plots" / exportfilename

export_png(p, filename=exportfilename)
shutil.move(str(source), str(destination))

show(p) 

In [55]:
# Renaming cols
# df = df3.copy()
# df = df.rename(columns=rename_col)
# 
# new_order = [0, 2, 4, 1, 3, 5]
# df = df.iloc[new_order]
# 
# # Extracting foldchange and cleaning up column names
# foldchange_pattern = r".*_mean"
# diffs_new = ["D0",  "100vv D0", "150vv D0", "D2", "100vv D2", "150vv D2"]
# primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]
# 
# ci95_upper_pattern = r".*_ci95_upper"
# ci95_lower_pattern = r".*_ci95_lower"
# ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
# ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]
# 
# 
# primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]
# 
# data = {
#     "CHIR Timing": diffs_new,
#     "CER1": df[primers[2]],
#     "CER1_upper": df[ci95_upper[2]],
#     "CER1_lower": df[ci95_lower[2]],
#     "LHX1": df[primers[3]],
#     "LHX1_upper": df[ci95_upper[3]],
#     "LHX1_lower": df[ci95_lower[3]],
#     "CYP26A": df[primers[4]],
#     "CYP26A_upper": df[ci95_upper[4]],
#     "CYP26A_lower": df[ci95_lower[4]],
# }
# primers = [re.sub(r"_mean$", "", s) for s in primers]
# source = ColumnDataSource(data=data)
# 
# p = figure(
#     x_range=diffs_new, y_range=(0.001, 500), height=350, width=650,
#     title="Relative fold change of DE marker genes (vs GAPDH) in WTC11 derived samples",
#     toolbar_location=None, tools="", y_axis_type="log"
# )
# 
# bar_width = 0.2
# p.vbar(
#     x=dodge("CHIR Timing", -0.3, range=p.x_range),
#     top="CER1", bottom=0.1, width=bar_width, source=source,
#     color=Bright6[0], legend_label="CER1"
# )
# p.vbar(
#     x=dodge("CHIR Timing", -0.1, range=p.x_range),
#     top="LHX1", bottom=0.1, width=bar_width, source=source,
#     color=Bright6[1], legend_label="LHX1"
# )
# 
# p.vbar(
#     x=dodge("CHIR Timing", 0.1, range=p.x_range),
#     top="CYP26A", bottom=0.1, width=bar_width, source=source,
#     color=Bright6[2], legend_label="CYP26A"
# )
# 
# cer1_err = Whisker(
#     base=dodge("CHIR Timing", -0.3, range=p.x_range),
#     upper='CER1_upper', lower='CER1_lower', source=source, level="overlay"
# )
# cer1_err.upper_head.size = 10
# cer1_err.lower_head.size = 10
# p.add_layout(cer1_err)
# 
# lhx1_err = Whisker(
#     base=dodge("CHIR Timing", -0.1, range=p.x_range),
#     upper='LHX1_upper', lower='LHX1_lower', source=source, level="overlay"
# )
# lhx1_err.upper_head.size = 10
# lhx1_err.lower_head.size = 10
# p.add_layout(lhx1_err)
# 
# 
# cyp_err = Whisker(
#     base=dodge("CHIR Timing", 0.1, range=p.x_range),
#     upper='CYP26A_upper', lower='CYP26A_lower', source=source, level="overlay"
# )
# cyp_err.upper_head.size = 10
# cyp_err.lower_head.size = 10
# p.add_layout(cyp_err)
# 
# p.y_range.start = 0.1
# p.x_range.range_padding = 0.1
# p.xaxis.major_label_orientation = 1
# p.xgrid.grid_line_color = None
# p.legend.location = "top_right"
# p.legend.orientation = "horizontal"
# p.xaxis.axis_label_text_font_style = "normal"
# p.yaxis.axis_label_text_font_style = "normal"
# 
# p.xaxis.axis_label = "iPSC to DE Differentiation Sample Days (Unperturbed vs Infected)"
# p.yaxis.axis_label = "Relative fold change wrt D0 (2^(-ΔΔCq))" 
# 
# 
# # # Export directory
# # exportfilename = "JR98-250306_TBD.png"
# # source = QPCRPATHS.ROOT/ "analysis" / exportfilename
# # destination = QPCRPATHS.DATADIR / experiment_id / "250306_qpcr_analysis" / "plots" / exportfilename
# # 
# # export_png(p, filename=exportfilename)
# # shutil.move(str(source), str(destination))
# 
# show(p) 

In [60]:
# Renaming cols
df = df3.copy()
df = df.rename(columns=rename_col)

# Extracting foldchange and cleaning up column names
foldchange_pattern = r".*_mean"
diffs_ori = ["D0", "D2", "R1 100vv D0", "R1 100vv D2", "R1 150vv D0", "R1 150vv D2", "R2 100vv D0", "R2 100vv D2", "R2 150vv D0", "R2 150vv D2"]
primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

ci95_upper_pattern = r".*_ci95_upper"
ci95_lower_pattern = r".*_ci95_lower"
ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]

data = {
    "CHIR Timing": diffs_ori,
    "OCT4": df[primers[0]],
    "OCT4_upper": df[ci95_upper[0]],
    "OCT4_lower": df[ci95_lower[0]],
    "NANOG": df[primers[1]],
    "NANOG_upper": df[ci95_upper[1]],
    "NANOG_lower": df[ci95_lower[1]],
}
primers = [re.sub(r"_mean$", "", s) for s in primers]
source = ColumnDataSource(data=data)

# Plotting the figure
p = figure(
    x_range=diffs_ori, y_range=(0, 2), height=350, width=350,
    title="Relative fold change of iPSC marker genes \n(vs GAPDH) in WTC11 derived samples",
    toolbar_location=None, tools=""
)

bar_width = 0.2
p.vbar(
    x=dodge("CHIR Timing", -0.1, range=p.x_range),
    top="OCT4", bottom=0.1, width=bar_width, source=source,
    color=Bright6[4], legend_label="OCT4"
)

p.vbar(
    x=dodge("CHIR Timing", 0.1, range=p.x_range),
    top="NANOG", bottom=0.1, width=bar_width, source=source,
    color=Bright6[5], legend_label="NANOG"
)

oct4_err = Whisker(
    base=dodge("CHIR Timing", -0.1, range=p.x_range),
    upper='OCT4_upper', lower='OCT4_lower', source=source, level="overlay"
)
oct4_err.upper_head.size = 3
oct4_err.lower_head.size = 3
p.add_layout(oct4_err)

nano_err = Whisker(
    base=dodge("CHIR Timing", 0.1, range=p.x_range),
    upper='NANOG_upper', lower='NANOG_lower', source=source, level="overlay"
)
nano_err.upper_head.size = 3
nano_err.lower_head.size = 3
p.add_layout(nano_err)

p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label = "iPSC to DE Differentiation Sample Days \n(Unperturbed vs Infected)"
p.yaxis.axis_label = "Relative fold change wrt D0 (2^(-ΔΔCq))" 

exportfilename = "JR98-250306_qpcr_r1_d0markers.png"
source = QPCRPATHS.ROOT/ "analysis" / exportfilename
destination = QPCRPATHS.DATADIR / experiment_id / "250306_qpcr_analysis" / "plots" / exportfilename

export_png(p, filename=exportfilename)
shutil.move(str(source), str(destination))

show(p) 